# Model Training

#### 1.1 Import Data and Required Packages

##### Importing Pandas,Numpy, Matplotlib, Seaborn amd Warnings Library

In [ ]:
# Basic Import
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Modelling
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression,Ridge,Lasso,LinearRegression
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

##### Import the csv Data as Pandas DataFrame

In [ ]:
loan_train = pd.read_csv('data/loan-train.csv')

In [ ]:
loan_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


##### Preparing X and Y Variable

In [ ]:
# Droping the Load_ID Columns since it is serial number only

loan_train.drop(['Loan_ID'], axis = 1 , inplace =True)

X = loan_train.drop(columns={'Loan_Status'},axis=1)
y = loan_train['Loan_Status']


In [ ]:
loan_train.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

##### Print the unique Categories

In [ ]:

print("Categories in 'Gender' variable are:",loan_train['Gender'].unique())
print()
print("Categories in 'Married' variable are:",loan_train['Married'].unique())
print()
print("Categories in 'Dependents' variable are:",loan_train['Dependents'].unique())
print()
print("Categories in 'Self_Employed' variable are:",loan_train['Self_Employed'].unique())
print()
print("Categories in 'Credit_History' variable are:",loan_train['Credit_History'].unique())

Categories in 'Gender' variable are: ['Male' 'Female' nan]

Categories in 'Married' variable are: ['No' 'Yes' nan]

Categories in 'Dependents' variable are: ['0' '1' '2' '3+' nan]

Categories in 'Self_Employed' variable are: ['No' 'Yes' nan]

Categories in 'Credit_History' variable are: [ 1.  0. nan]


##### Filling Missing Nan Values with mean, mode and replacing with greater value.

In [ ]:
loan_train['Gender'].fillna(loan_train['Gender'].mode()[0], inplace=True)

loan_train['Dependents'].fillna(loan_train['Dependents'].mode()[0], inplace=True)

loan_train['Self_Employed'].fillna(loan_train['Self_Employed'].mode()[0], inplace=True)

loan_train['Married'].fillna(loan_train['Married'].mode()[0], inplace=True)

loan_train['LoanAmount'].fillna(loan_train['LoanAmount'].mean(), inplace=True)

loan_train['Loan_Amount_Term'].fillna(loan_train['Loan_Amount_Term'].mean(), inplace=True)

loan_train['Credit_History'].replace((np.NaN,'1'), inplace=True)

In [ ]:
loan_train.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

##### Insight 
- No Null values Found

In [ ]:
loan_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             614 non-null    object 
 1   Married            614 non-null    object 
 2   Dependents         614 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      614 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         614 non-null    float64
 8   Loan_Amount_Term   614 non-null    float64
 9   Credit_History     614 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


In [ ]:
loan_train.head(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.000000,360.0,1.0,Urban,Y
6,Male,Yes,0,Not Graduate,No,2333,1516.0,95.000000,360.0,1.0,Urban,Y
7,Male,Yes,3+,Graduate,No,3036,2504.0,158.000000,360.0,0.0,Semiurban,N
8,Male,Yes,2,Graduate,No,4006,1526.0,168.000000,360.0,1.0,Urban,Y
9,Male,Yes,1,Graduate,No,12841,10968.0,349.000000,360.0,1.0,Semiurban,N


In [ ]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,cat_features),
        ('StandardScaler',numeric_transformer,num_features)
    ]
)

# Converting Categorical Feature into Numerical Feature
# loan_train.Loan_Status = loan_train.Loan_Status.replace({"Y": 1, "N" : 0},inplace=True)

In [ ]:
num_features,cat_features

(Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
        'Loan_Amount_Term', 'Credit_History'],
       dtype='object'),
 Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
        'Property_Area'],
       dtype='object'))

In [ ]:
loan_train.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
# Separate dataset into train and test

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((491, 24), (123, 24))

#### Create an Evaluate Function to give all metrics after model training

In [ ]:
X_train = pd.DataFrame(X_train)
X_train[:].fillna(X_train[:].mean(), inplace=True)
X_train

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.097728,0.215042,1.386749,0.276642,0.007107
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.400628,0.437351,-0.180228,0.276642,0.432861
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.138685,0.025225,0.743587,0.276642,0.432861
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.543975,0.522854,-0.215309,0.276642,0.432861
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.202727,-0.554487,-0.566125,0.276642,-2.310212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,-0.578051,0.086787,-0.577819,0.276642,0.432861
487,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.985170,-0.169381,0.918995,0.276642,0.432861
488,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.354921,-0.554487,-1.361307,0.276642,0.432861
489,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.760729,-0.554487,0.027012,-1.567636,0.432861


In [ ]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
dtype: int64

In [ ]:
X_train,y_train

(      0    1    2    3    4    5    6    7    8    9   ...   14   15   16   
 0    0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  \
 1    0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2    0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 3    0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 4    0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 ..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 486  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  ...  1.0  0.0  0.0   
 487  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 488  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 489  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 490  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 
       17   18        19        20        21        22        

In [ ]:
X_test,y_test

(array([[ 0.        ,  1.        ,  0.        , ...,  0.95407625,
          0.27664167,  0.43286074],
        [ 0.        ,  1.        ,  0.        , ..., -0.19192142,
          0.27664167,         nan],
        [ 0.        ,  1.        ,  0.        , ...,  0.13550648,
          0.27664167,  0.43286074],
        ...,
        [ 0.        ,  1.        ,  0.        , ..., -0.2737784 ,
          2.12091945,  0.43286074],
        [ 0.        ,  1.        ,  0.        , ..., -0.43749235,
          0.27664167,  0.43286074],
        [ 0.        ,  1.        ,  0.        , ...,  4.01786593,
         -2.48977501,  0.43286074]]),
 350    1
 377    1
 163    1
 609    1
 132    1
       ..
 231    1
 312    1
 248    1
 11     1
 333    1
 Name: Loan_Status, Length: 123, dtype: int64)

In [ ]:
X_test = pd.DataFrame(X_test)
X_test[:].fillna(X_test[:].mean(), inplace=True)
X_test

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.602802,-0.554487,0.954076,0.276642,0.432861
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.179136,-0.554487,-0.191921,0.276642,-0.028364
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.202563,-0.059594,0.135506,0.276642,0.432861
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.410130,-0.554487,-0.881859,0.276642,0.432861
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.439946,-0.554487,-0.893553,0.276642,0.432861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,-0.276448,-0.554487,-1.220981,-2.489775,0.432861
119,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.474513,-0.554487,-1.057267,0.276642,0.432861
120,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.413079,0.075843,-0.273778,2.120919,0.432861
121,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,-0.475660,0.074817,-0.437492,0.276642,0.432861


In [ ]:
def evalute_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true,predicted)
    return mae,rmse,r2_square

In [ ]:
models = {
    'Linear Regressor' : LinearRegression(),
    'Logistic Resgressor' : LogisticRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'K-Neighbors Regressor' : KNeighborsRegressor(),
    'Decision Tree' : DecisionTreeRegressor(),
    'Random Forest Regressor' : RandomForestRegressor(),
    'Ada Boost' : AdaBoostRegressor(),
    'XGBoost Regressor' : XGBRegressor(),
    'CatBoosting Regression' : CatBoostRegressor(verbose=False)
 }

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train) # Train Model

    # Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test datatest
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for Training Set')
    print("- Root Mean Squared Error :{:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error : {:.4f}".fromat(model_train_mae))
    print("- R2 Score : {:.4f}".format(model_train_r2))

    print('--------------------------------------')

    print('Model Performance for Test Set')
    print("- Root Mean Squared Error :{:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error : {:.4f}".fromat(model_test_mae))
    print("- R2 Score : {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

NameError: name 'evaluate_model' is not defined